This is the code that I used in order to get the rating and overview data for this dataset. In case you want to tamper with this code to add extra data as you need, feel free to do just that. However, it takes a while to get the data, which is why I only used the first 2000 datasets. I have added the dataset with 11000 movie titles in this notebook.

In [ ]:
#Import the neccessary modules
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

As you can see in this line of code, I searched through the first 2000 titles and overview of the dataset. You can tamper this as you wish

In [ ]:
number_titles = 2000
movie_text = pd.read_csv('movies_metadata.csv', low_memory=False)[:number_titles][['original_title','overview']]

The next batch of code will be the whole code that I used to make this, in case you want to change number_titles

In [ ]:
#Import neccessary modules
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

#Read the first 2000 parts of the dataset
number_titles = 2000
movie_text = pd.read_csv('movies_metadata.csv', low_memory=False)[:number_titles][['original_title','overview']]
movie_text = movie_text.fillna("none")
#Create a list with the movie overviews(summaries)
movie_overview = list(np.array(movie_text['overview']))

#Begin making the lists that will be used for the parsing
titles = [title for title in movie_text['original_title']]
ratings = []
for title in titles:
    #Format the title so that the rotten tomatoes website can read it
    title = '_'.join(title.split())
    #Get the URL based off of the title, and use BeautifulSoup and requests to parse through it in html
    theURL = 'https://www.rottentomatoes.com/m/{}'.format(title)
    google_page = requests.get(theURL)
    soup_google = BeautifulSoup(google_page.text, "html.parser")
    #In the span class, there is the Rotten Tomatoes rating, and this line of code finds it
    rating_html = soup_google.find_all('span', {'class':'mop-ratings-wrap__percentage'})
    #There are some movies that Rotten Tomatoes hasn't rated. This try except code will append 'no rating' if that's the case
    canDo = True
    try:
        the_split = str(rating_html[0]).split('>')[1]
    except:
        canDo = False
        ratings.append('no rating')
    #If there was no exception, the code then appends the actual rating to the list
    if canDo:
        the_num = int(the_split.rstrip('\n').split('%')[0])
        ratings.append(the_num)
        
#Make everything into numpy arrays for the pd DataFrame to understand        
titles = np.array(titles)
ratings = np.array(ratings)
movie_overview = np.array(movie_overview)

#Finally, make everything into a pandas datafram and convert that to a csv file
dataset = pd.DataFrame({
    "title":titles,
    "RT rating":ratings,
    "overview":movie_overview
})
dataset.to_csv('movie_rating_overview.csv', index=False)

In case you want to add to the original dataset so you don't have to waste time going through the first 2000 titles that I've already done in the dataset, here's what you can do:

In [ ]:
original_dataset = pd.read_csv('movie_rating_overview_2.csv') #My dataset
new_dataset = pd.DataFrame({
    "title":titles,
    "RT rating":ratings,
    "overview":movie_overview
})
original_dataset = original_dataset.append(new_dataset)
original_dataset.to_csv('better_movie_rating_overview.csv', index=False)

However, if you get blocked by the Rotten Tomatoes website for parsing through their website, another option is to parse through the google website itself:

In [ ]:
#This code is inside the for loop:
theURL = 'https://www.google.com/search?safe=strict&rlz=1C1CHBF_enUS851US851&sxsrf=ALeKk02JvMu2BU4Qp3tBW4ULuXTXIRqljA%3A1606182990021&ei=Tmi8X7ZpxIq1BdOukcgI&q=rotten+tomatoes+score{}&oq=infinity+war+movie&gs_lcp=CgZwc3ktYWIQAzIHCC4QDRCTAjIECAAQDTIECAAQDTIECAAQDTIGCAAQBxAeMgQIABANMgQIABANMgQIABANMgQIABANMgQIABANOgcILhBDEJMCOgQIABBDOgUIABCxAzoHCAAQsQMQQzoECC4QQzoCCAA6CgguEBQQhwIQkwI6BAguEApQqb4rWJiALGDMgCxoAHABeACAAYYCiAGCDZIBBjAuMTAuMpgBAKABAaoBB2d3cy13aXrAAQE&sclient=psy-ab&ved=0ahUKEwi2yMH0iZrtAhVERa0KHVNXBIkQ4dUDCA0&uact=5'.format(title)
google_page = requests.get(theURL)
soup_google = BeautifulSoup(google_page.text, "html.parser")
rating_html = soup_google.find_all('span', {'class':'XLloXe AP7Wnd'})
canDo = True
try:
    #This is the way in order to get the rotten tomatoes score that pops up when you search it up
    theNum = int(str(soup_google).split('%</span>')[0].split('Jd">')[1]) 
except:
    canDo = False
    ratings.append('no rating')
if canDo:
    ratings.append(theNum)

This is the full code for parsing through the Google Website, just in case you need it:

In [ ]:
#Import neccessary modules
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

#Read the first 4000 parts of the dataset
the_amount = 4000
movie_text = pd.read_csv('movies_metadata.csv', low_memory=False)[:the_amount][['original_title','overview']]
movie_text = movie_text.fillna("none")
#Create a list with the movie overviews(summaries)
movie_overview = list(np.array(movie_text['overview']))

#Begin making the lists that will be used for the parsing
titles = [title for title in movie_text['original_title']]
ratings = []
counter = 0
for title in titles:
    title = title.lower()
    title = '_'.join(title.split())
    #Use the Google URL and parse through it
    theURL = 'https://www.google.com/search?safe=strict&rlz=1C1CHBF_enUS851US851&sxsrf=ALeKk02JvMu2BU4Qp3tBW4ULuXTXIRqljA%3A1606182990021&ei=Tmi8X7ZpxIq1BdOukcgI&q=rotten+tomatoes+score{}&oq=infinity+war+movie&gs_lcp=CgZwc3ktYWIQAzIHCC4QDRCTAjIECAAQDTIECAAQDTIECAAQDTIGCAAQBxAeMgQIABANMgQIABANMgQIABANMgQIABANMgQIABANOgcILhBDEJMCOgQIABBDOgUIABCxAzoHCAAQsQMQQzoECC4QQzoCCAA6CgguEBQQhwIQkwI6BAguEApQqb4rWJiALGDMgCxoAHABeACAAYYCiAGCDZIBBjAuMTAuMpgBAKABAaoBB2d3cy13aXrAAQE&sclient=psy-ab&ved=0ahUKEwi2yMH0iZrtAhVERa0KHVNXBIkQ4dUDCA0&uact=5'.format(title)
    google_page = requests.get(theURL)
    soup_google = BeautifulSoup(google_page.text, "html.parser")
    rating_html = soup_google.find_all('span', {'class':'XLloXe AP7Wnd'})
    #Use a try and except to see if google displays the rotten tomatoes score or not
    canDo = True
    try:
        theNum = int(str(soup_google).split('%</span>')[0].split('Jd">')[1])
    except:
        canDo = False
        ratings.append('no rating')
    #If it can, append the number to the ratings array
    if canDo:
        ratings.append(theNum)
        
#Make everything into numpy arrays for the pd DataFrame to understand        
titles = np.array(titles)
ratings = np.array(ratings)
movie_overview = np.array(movie_overview)

#Finally, make everything into a pandas datafram and convert that to a csv file
dataset = pd.DataFrame({
    "title":titles,
    "RT rating":ratings,
    "overview":movie_overview
})
dataset.to_csv('movie_rating_overview.csv', index=False)